### Run in collab
<a href="https://colab.research.google.com/github/racousin/data_science_practice/blob/master/website/public/modules/data-science-practice/module9/exercise/module9_exercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install swig==4.2.1
!pip install gymnasium==1.2.0

In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
from tqdm.auto import trange, tqdm

print("gymnasium:", gym.__version__)
print("numpy:", np.__version__)

gymnasium: 1.2.0
numpy: 2.0.2


# module9_exercise2 : ML - Arena <a href="https://ml-arena.com/viewcompetition/5" target="_blank"> FrozenLake Competition</a>

### Objective
Get at list an agent running on ML-Arena <a href="https://ml-arena.com/viewcompetition/5" target="_blank"> FrozenLake Competition</a> with mean reward upper than 0.35 (ie 35%)


You should submit an agent file named `agent.py` with a class `Agent` that includes at least the following attributes:

### Description

The game starts with the player at location [0,0] of the frozen lake grid world with the goal located at far extent of the world [7,7].

Holes in the ice are distributed in set locations.

The player makes moves until they reach the goal or fall in a hole.

Each run will consist of 10 attempts to cross the ice. The reward will be the total amount accumulated during those trips. For example, if your agent reaches the goal 3 times out of 10, its reward will be 3.

The environment is based on :

In [3]:
import numpy as np
from tqdm.auto import trange
import gymnasium as gym

def make_env(seed: int = 0, is_slippery: bool = True):
    env = gym.make('FrozenLake-v1', map_name="8x8", is_slippery=is_slippery)
    env.action_space.seed(seed)
    return env

def _assert_gym_order():
    import gymnasium as gym
    side = 8
    def idx_to_rc(s): return divmod(s, side)
    def rc_to_idx(r,c): return r*side + c

    env = gym.make("FrozenLake-v1", map_name="8x8", is_slippery=False)

    # IMPORTANT: reset AVANT tout step()
    _, _ = env.reset(seed=0)

    s0 = rc_to_idx(3,3)
    # on force la position après reset
    env.unwrapped.s = s0

    expected = {0:(0,-1), 1:(+1,0), 2:(0,+1), 3:(-1,0)}
    for a in range(4):
        env.unwrapped.s = s0
        # le env a bien été reset; on peut step
        s1, *_ = env.step(a)
        (r0,c0) = idx_to_rc(s0); (r1,c1) = idx_to_rc(s1)
        assert (r1-r0, c1-c0) == expected[a], f"Action {a} inattendue"
    env.close()
    print("✔️ Gym order vérifié: 0←,1↓,2→,3↑")


class QLearner:
    def __init__(self, n_states: int, n_actions: int, seed: int = 0):
        self.q = np.zeros((n_states, n_actions), dtype=np.float32)
        self.n_actions = n_actions
        self.rng = np.random.default_rng(seed)
    def _argmax_tiebreak(self, row):
        m = row.max(); idx = np.flatnonzero(row == m)
        return int(self.rng.choice(idx))
    def policy_eps_greedy(self, state: int, epsilon: float) -> int:
        if self.rng.random() < epsilon:
            return int(self.rng.integers(0, self.n_actions))  # 0..3
        return self._argmax_tiebreak(self.q[state])
    def q_update(self, s, a, r, s_next, done: bool, alpha: float, gamma: float):
        best_next = 0.0 if done else float(np.max(self.q[s_next]))
        self.q[s, a] += alpha * (r + gamma * best_next - self.q[s, a])

def train_q_learning(
    episodes=200_000, max_steps=200, gamma=0.995,
    alpha_start=0.5, alpha_end=0.05,
    eps_start=1.0, eps_end=0.01, eps_decay_steps=120_000,
    seed=2025
):
    _assert_gym_order()

    env = make_env(seed, is_slippery=True)
    # reset immédiat pour initialiser l’état interne du wrapper
    _ , _ = env.reset(seed=seed)

    agent = QLearner(n_states=env.observation_space.n, n_actions=env.action_space.n, seed=seed)
    rewards = np.zeros(episodes, dtype=np.float32)

    for ep in trange(episodes, desc="Training"):
        epsilon = (eps_start + min(1.0, ep/max(eps_decay_steps,1))*(eps_end-eps_start))
        alpha   = (alpha_start + (ep/episodes)*(alpha_end-alpha_start))

        # reset en début d’épisode (critique)
        s, _ = env.reset(seed=seed + ep)
        ep_rew = 0.0

        for _ in range(max_steps):
            a = agent.policy_eps_greedy(s, epsilon)
            s_next, r, terminated, truncated, _ = env.step(a)

            done_bootstrap = bool(terminated)  # ne coupe pas sur troncature
            agent.q_update(s, a, r, s_next, done_bootstrap, alpha, gamma)

            s = s_next
            ep_rew += r
            if terminated or truncated:
                break

        rewards[ep] = ep_rew

    env.close()
    return agent, rewards

def evaluate_slippery(agent, episodes=2000, max_steps=200, seed=777):
    env = make_env(seed, is_slippery=True)
    wins = 0
    for ep in range(episodes):
        s,_ = env.reset(seed=seed + ep)
        for _ in range(max_steps):
            a = int(np.argmax(agent.q[s]))
            s,r,term,trunc,_ = env.step(a)
            if term or trunc:
                wins += r
                break
    env.close()
    return wins / episodes




In [4]:
agent, rewards = train_q_learning(episodes=10_000, seed=2025)
print("Last-1000 mean:", float(rewards[-1000:].mean()))
print("With slippery (2000 eps):", evaluate_slippery(agent, episodes=2000))



✔️ Gym order vérifié: 0←,1↓,2→,3↑


Training:   0%|          | 0/10000 [00:00<?, ?it/s]

Last-1000 mean: 0.003000000026077032
With slippery (2000 eps): 0.4775


In [5]:
def small_search():
    configs = [
        {"gamma": 0.99,  "alpha_start": 0.5, "eps_decay_steps": 40_000},
        {"gamma": 0.995, "alpha_start": 0.5, "eps_decay_steps": 60_000},
        {"gamma": 0.997, "alpha_start": 0.4, "eps_decay_steps": 80_000},
    ]

    results = []
    for i, cfg in enumerate(configs, 1):
        print(f"\n[{i}/{len(configs)}] test cfg={cfg}")

        agent, rewards = train_q_learning(
            episodes=120_000,   # un peu + long que 80k pour stabiliser
            gamma=cfg["gamma"],
            alpha_start=cfg["alpha_start"],
            alpha_end=0.05,
            eps_start=1.0,
            eps_end=0.01,
            eps_decay_steps=cfg["eps_decay_steps"],
            seed=1000 + i,
        )

        score_slip = evaluate_slippery(agent, episodes=2000)


        results.append((cfg, score_slip))

        print(f"🎯 Slippery score: {score_slip:.4f}")

    return results


In [6]:
results = small_search()
best_cfg, best_score = max(results, key=lambda x: x[1])
print("\nBest config:", best_cfg)
print("Best slippery score:", best_score)



[1/3] test cfg={'gamma': 0.99, 'alpha_start': 0.5, 'eps_decay_steps': 40000}
✔️ Gym order vérifié: 0←,1↓,2→,3↑


Training:   0%|          | 0/120000 [00:00<?, ?it/s]

🎯 Slippery score: 0.6150

[2/3] test cfg={'gamma': 0.995, 'alpha_start': 0.5, 'eps_decay_steps': 60000}
✔️ Gym order vérifié: 0←,1↓,2→,3↑


Training:   0%|          | 0/120000 [00:00<?, ?it/s]

🎯 Slippery score: 0.5895

[3/3] test cfg={'gamma': 0.997, 'alpha_start': 0.4, 'eps_decay_steps': 80000}
✔️ Gym order vérifié: 0←,1↓,2→,3↑


Training:   0%|          | 0/120000 [00:00<?, ?it/s]

🎯 Slippery score: 0.5585

Best config: {'gamma': 0.99, 'alpha_start': 0.5, 'eps_decay_steps': 40000}
Best slippery score: 0.615


In [7]:
agent, rewards = train_q_learning(
    episodes=300_000,
    gamma=best_cfg["gamma"],
    alpha_start=best_cfg["alpha_start"], alpha_end=0.05,
    eps_start=1.0, eps_end=0.01, eps_decay_steps=best_cfg["eps_decay_steps"],
    seed=2025
)

print("Last-10k average:", rewards[-10_000:].mean())
print("Final slippery score:", evaluate_slippery(agent, episodes=5000))


✔️ Gym order vérifié: 0←,1↓,2→,3↑


Training:   0%|          | 0/300000 [00:00<?, ?it/s]

Last-10k average: 0.5713
Final slippery score: 0.6328


In [8]:
np.save("q_table.npy", agent.q)
print("✅ q_table.npy saved")


✅ q_table.npy saved


### Before submit
Test that your agent has the right attributes

# **SUBMISSION**

---



In [9]:
# Génération d'un agent.py autonome pour ML Arena
import numpy as np, json

q = np.load("q_table.npy")
q_list = np.asarray(q, dtype=float).round(6).tolist()

agent_code = f"""
import random

class Agent:
    def __init__(self, env):
        self.env = env
        self.Q = {json.dumps(q_list)}
        self.nS = len(self.Q)
        self.nA = len(self.Q[0]) if self.nS>0 else env.action_space.n
        self.rng = random.Random(0)

    def choose_action(self, observation, reward=0.0, terminated=False, truncated=False, info=None):
        s = int(observation)
        if s < 0 or s >= self.nS:
            return self.env.action_space.sample()
        row = self.Q[s]
        m = max(row)
        candidates = [a for a, v in enumerate(row) if v == m]
        return self.rng.choice(candidates)
"""

with open("agent.py", "w") as f:
    f.write(agent_code)

print("✅ agent.py autonome généré !")


✅ agent.py autonome généré !


In [12]:
from google.colab import files
files.download("agent.py")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
#======= TEST LOCAL ===========

import gymnasium as gym
from agent import Agent

env = gym.make("FrozenLake-v1", map_name="8x8", is_slippery=True)
agent = Agent(env)

obs, _ = env.reset()
reward, terminated, truncated, info = 0, False, False, None

while not (terminated or truncated):
    action = agent.choose_action(obs, reward, terminated, truncated, info)
    obs, reward, terminated, truncated, info = env.step(action)

print(f"✅ Episode terminé — reward: {reward}")


✅ Episode terminé — reward: 0.0
